In [1]:
import pandas as pd
import sqlite3
from os import path
import json
from lxml import html
import requests
import os
import datetime
from IPython.display import clear_output

bdPath = path.join("C:/", "Users", "Giovani", "Documents", "GitHub","topicosIC", "Em Grupo", "Trabalho Final", "BD", "bd.db")
nicksPath = path.join("..", "baseDeNicks.csv")

jogadoresPath = path.join("jogadores.sql")
campeoesJogadorPath = path.join("campeoesJogador.sql")
partidasPath = path.join("partidas.sql")
jogadoresPartidaPath = path.join("jogadoresPartida.sql")
logPath = path.join("nicksAdicionados.log")

In [2]:
bd = sqlite3.connect(bdPath) 

#Adiciona Elos
#elos = open('elos.sql', 'r').read()
#bd.executescript(elos)

#Adiciona Campeoes
#Criando inserts
#campeoesJson = json.load(open('lol-champions.json', 'rb'))
#n = len(campeoesJson)-1
#while (n>-1):
#    print("INSERT INTO campeao('campeaoId', 'nome') VALUES("+campeoesJson[n]['key']+", \""+campeoesJson[n]['name']+"\""+");") 
#    n=n-1
#campeoes = open('campeoes.sql', 'r').read()
#bd.executescript(campeoes)

#elo = pd.read_sql('SELECT * FROM elo', bd)
#campeoes = pd.read_sql('SELECT * FROM campeao', bd)

bd.close()

In [2]:
#Obtem informações do jogador e gera inserts pro bd
def getJogadorData(nick):
    baseUrl = "http://br.op.gg"
    playerUrl = "/summoner/userName="

    urlMontada = baseUrl+playerUrl+nick
    request = requests.get(urlMontada)
    html2 = html.fromstring(request.content)
    getData = html2.xpath('//div[@class="GameListContainer"]')
    if getData==[]:
        return -1
    summonerId = getData[0].attrib['data-summoner-id']

    elo = str(html2.xpath('//div/span[@class="tierRank"]/text()')).replace('[', '').replace(']', '').replace('\'', '')
    if elo=="":
        elo = "Unranked"
    pdl = str(html2.xpath('//div/span[@class="LeaguePoints"]/text()')).replace('[', '').replace(']', '').replace('\\n', '').replace('\\t', '').replace('\'', '').replace(' LP', '').replace(',', '')
    vitorias = str(html2.xpath('//span/span[@class="wins"]/text()')).replace('[', '').replace(']', '').replace('W', '').replace('L', '').replace('\'', '').replace(' LP', '').replace(',', '')
    derrotas = str(html2.xpath('//span/span[@class="losses"]/text()')).replace('[', '').replace(']', '').replace('W', '').replace('L', '').replace('\'', '').replace(' LP', '').replace(',', '')
    winrate = str(html2.xpath('//span/span[@class="winratio"]/text()')).replace('[', '').replace(']', '').replace('%', '').replace('Win Ratio ', '').replace('\'', '').replace(' LP', '')
    
    #TODO: Corrigir pois posições são dinamicamente colocadas após carregamento da página
    #posicoes = str(html2.xpath('//ul[@class="Content"]/li/div[@class="PositionStatContent"]/div/text()')).replace('[', '').replace(']', '').replace('\'', '').replace(' ', '').split(',')
    #posicao1 = posicoes[0]
    #posicao2 = posicoes[1]
    posicao1 = ""
    posicao2 = ""
    
    #DEBUGMODE
    '''
    print("Nick: " + nick + 
          "\nID: " + summonerId + 
          "\nElo: " + elo + 
          " - " + pdl + " PDL" +
          "\nVitórias: " + vitorias + 
          "\nDerrotas: " + derrotas + 
          "\nTaxa de Vitória: " + winrate + "%" +
          "\nPosição 1: " + posicao1 + 
          "\nPosição 2: " + posicao2)
    '''
    
    jogadoresSql = open(jogadoresPath, "a")
    jogadoresSql.write("INSERT INTO jogador('summonerId', 'nick', 'elo', 'pdl', 'vitorias', 'derrotas', 'winrate', 'posicao1', 'posicao2') VALUES("+summonerId+","+"\'"+nick+"\'"+","+"\'"+elo+"\'"+","+pdl+","+vitorias+","+derrotas+","+winrate+","+"\'"+posicao1+"\'"+","+"\'"+posicao2+"\');\n")
    jogadoresSql.close()
    return summonerId 

In [3]:
#Obtem informações dos campeoes do jogador e gera inserts pro bd
def getCampeoesJogadorData(summonerId):
    baseUrl = "http://br.op.gg"
    champsUrl = "/summoner/champions/ajax/champions.rank/summonerId="
    season = "&season=12"

    urlMontada = baseUrl+champsUrl+summonerId+season
    request = requests.get(urlMontada)
    while request.status_code!=200:
        request = requests.get(urlMontada)
    html2 = html.fromstring(request.content)

    nCampeoes = html2.xpath('//tr[@class="Row TopRanker"]')
    for i in range(len(nCampeoes)):
        pref = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="Rank Cell"]/text()')[i]
        nome = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="ChampionName Cell"]/a/text()')[i]
        vitorias = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="RatioGraph Cell"]/div/div/div[@class="Text Left"]/text()')    
        if i>=len(vitorias):
            vitorias = '0'
        else:
            vitorias = vitorias[i].replace("W", "")

        derrotas = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="RatioGraph Cell"]/div/div/div[@class="Text Right"]/text()')
        if i>=len(derrotas):
            derrotas = '0'
        else:
            derrotas = derrotas[i].replace("L", "")
        
        partidas = str(int(vitorias)+int(derrotas))
        winrate = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="RatioGraph Cell"]/div/span/text()')[i].replace("%", "")
        kda = html2.xpath('//tr[@class="Row TopRanker"]/td[contains(@class, \'KDA \')]')[i].attrib['data-value']

        #TODO: Corrigir separação de cada valor pois o site é mal estruturado nesse ponto
        #values = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="Value Cell"]/text()')[i].split(' ')
        #values = str(values).replace('\\n', "").replace('\\t', "").replace('[', "").replace(']', "").replace(',', "").replace('\'', "")
        #values = str(values).splitlines()
        #ouro = values[0]
        #cs = values[1]
        #danoCausado = values[4]
        #danoRecebido = values[5]

        #DEBUGMODE
        '''
        print("ID: " + summonerId + 
              " Campeão: " + nome + 
              " Preferência: " + pref + 
              " Partidas: " + partidas + 
              " Vitórias: " + vitorias + 
              " Derrotas: " + derrotas + 
              " Taxa de Vitória: " + winrate + "%" +
              " KDA: " + kda)
        '''

        #print("\nINSERT INTO campeao_jogador('summonerId', 'campeaoNome', 'campeaoPref', 'partidas', 'vitorias', 'derrotas', 'winrate', 'kda', 'ouro', 'cs', 'danoCausado', 'danoRecebido') VALUES("
             # +summonerId+","+"\'"+nome+"\'"+","+pref+","+vitorias+derrotas+","+vitorias+","+derrotas+","+winrate+","+","+kda+","+ouro+","+cs+","+danoCausado+","+danoRecebido+");")

        campeoesJogadorSql = open(campeoesJogadorPath, "a")
        campeoesJogadorSql.write("INSERT INTO campeao_jogador('summonerId', 'campeaoNome', 'campeaoPref', 'partidas', 'vitorias', 'derrotas', 'winrate', 'kda', 'ouro', 'cs', 'danoCausado', 'danoRecebido') VALUES("
          +summonerId+","+"\""+nome+"\""+","+pref+","+partidas+","+vitorias+","+derrotas+","+winrate+","+kda+","+'0'+","+'0'+","+'0'+","+'0'+");\n")
        campeoesJogadorSql.close()

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
    
#Funcao de apoio - converte nick em id
'''
def findIdByName(summonerNome):
    baseUrl = "http://br.op.gg"
    playerUrl = "/summoner/userName="   

    urlMontada = baseUrl+playerUrl+summonerNome
    request = requests.get(urlMontada)
    html2 = html.fromstring(request.content)
    getData = html2.xpath('//div[@class="GameListContainer"]')
    summonerId = getData[0].attrib['data-summoner-id']

    return summonerId
'''
def findIdByName(summonerNome):
    bd = sqlite3.connect(bdPath) 
    cursor = bd.execute("SELECT (summonerId) FROM 'jogador' where nick=\""+summonerNome+"\"")
    data=cursor.fetchall()
    if len(data)==0:
        cursor = bd.execute("SELECT (summonerId) FROM 'jogador' where nick=\" "+summonerNome+"\"")
        data=cursor.fetchall()
        if len(data)==0:
            return "-1"
        else:
            return str(data[0][0])   
    else:
        return str(data[0][0])
    bd.close()
    
#Obtem informações das partidas do jogador e gera inserts pro bd
def getPartidasJogadorData(summonerId):   
    baseUrl = "http://br.op.gg"
    champsUrl = "/summoner/matches/ajax/averageAndList/startInfo=0&summonerId="
    partidaUrl = "/summoner/matches/ajax/detail/gameId="
    seasonType = "&type=soloranked&season=12"
    
    urlMontada = baseUrl+champsUrl+summonerId+seasonType
    driver = webdriver.Chrome()
    driver.get(urlMontada)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    listaPartidas = soup.findAll("div", {"class": "GameItemWrap"})
    
    import json
    try:
        js = json.loads(soup.text)
    except ValueError:
        return -1
    
    teste = BeautifulSoup(js["html"], 'html.parser')
    listaPartidas = teste.findAll("div", {"class": "GameItemWrap"})

    campeoes = teste.findAll("div", {"class": "ChampionName"})
    kdas = teste.findAll("span", {"class": "KDARatio"})
    kdas.pop(0)
    desc = teste.findAll("td", {"class": "SummonerName"})    
    partidas = teste.findAll("div", {"class": "GameItem"})
    

    for i in range(len(listaPartidas)):
    #for i in range(1):
        sumId = partidas[i].get('data-summoner-id')
        gameTime = partidas[i].get('data-game-time')
        gameId = partidas[i].get('data-game-id')
        
        #Informações da partida
        urlMontada2 = baseUrl+partidaUrl+gameId+"&summonerId="+sumId+"&gameTime="+gameTime
        request = requests.get(urlMontada2)
        html2 = html.fromstring(request.text)
                
        values = ['0','0','0','0','0','0']
        summary = html2.xpath('//div[contains(@class, \'Team\')]/div[@class="ObjectScore"]/text()')
        n=0
        for k in summary[1::2]:
            values[n] = k.strip()
            n = n + 1
        baroesBlue = values[0]
        dragoesBlue = values[1]
        torresBlue = values[2]
        baroesRed = values[3]
        dragoesRed = values[4]
        torresRed = values[5]
        

        dadosBlue = html2.xpath('//div[@class="text graph--data graph--data__left"]/text()')
        dadosRed = html2.xpath('//div[@class="text graph--data graph--data__right"]/text()')        
        
        if len(dadosBlue)>1:
            killsBlue = dadosBlue[0]
            killsRed = dadosRed[0]
            ouroBlue = dadosBlue[1]
            ouroRed = dadosRed[1]
        else:
            killsBlue = '0'
            killsRed = '0'
            ouroBlue = '0'
            ouroRed = '0'           
        
        gameResult = html2.xpath('//th[@class="HeaderCell"]/span[@class="GameResult"]/text()')
        if gameResult[0].replace("\t", "").replace("\n", "") == "Victory":
            if html2.xpath('//tr/th[@class="HeaderCell"]/text()')[1].replace("\t", "").replace("\n", "") == "(Blue Team)":
                vitoria = "0"
            else:
                vitoria = "1"
        elif gameResult[0].replace("\t", "").replace("\n", "") == "Defeat":
                if html2.xpath('//tr/th[@class="HeaderCell"]/text()')[1].replace("\t", "").replace("\n", "") == "(Blue Team)":
                    vitoria = "1"
                else:
                    vitoria = "0"
        else:
            if gameResult[0].replace("\t", "").replace("\n", "") == "" or gameResult[1].replace("\t", "").replace("\n", "") == "":
                vitoria = "-1"
                
        #Remakes n importam para a base
        if vitoria=='-1':
            continue    
            
            
        #DEBUGMODE PARTIDA  
        '''
        print("GameID: " + gameId  + 
              " Vitória: " + vitoria +              
              " BaroesBlue: " + baroesBlue +
              " DragoesBlue: " + dragoesBlue  + 
              " TorresBlue: " + torresBlue  + 
              " BaroesRed: " + baroesRed + 
              " DragoesRed: " + dragoesRed  + 
              " TorresRed: " + torresRed  + 
              " KillsBlue: " + killsBlue  + 
              " KillsRed: " + killsRed  + 
              " OuroBlue: " + ouroBlue  + 
              " OuroRed: " + ouroRed)
        '''

        #print("INSERT INTO partida('gameId', 'vitoria', 'baroesBlue', 'dragoesBlue', 'torresBlue', 'baroesRed', 'dragoesRed', 'torresRed', 'killsBlue', 'killsRed', 'ouroBlue', 'ouroRed') VALUES("
        #      +gameId+","+vitoria+","+baroesBlue+","+dragoesBlue+","+torresBlue+","+baroesRed+","+dragoesRed+","+torresRed+","+killsBlue+","+killsRed+","+ouroBlue+","+ouroRed+");\n")
        partidasSql = open(partidasPath, "a")
        partidasSql.write("INSERT INTO partida('gameId', 'vitoria', 'baroesBlue', 'dragoesBlue', 'torresBlue', 'baroesRed', 'dragoesRed', 'torresRed', 'killsBlue', 'killsRed', 'ouroBlue', 'ouroRed') VALUES("
              +gameId+","+vitoria+","+baroesBlue+","+dragoesBlue+","+torresBlue+","+baroesRed+","+dragoesRed+","+torresRed+","+killsBlue+","+killsRed+","+ouroBlue+","+ouroRed+");\n")
        partidasSql.close()

        
        
        #Jogadores da partida
        html3 = html.fromstring(request.text)
        nicks = html3.xpath('//td[@class="SummonerName Cell"]/a/text()') 
        campeoes = html2.xpath('//tr[contains(@class, \'Row\')]/td/a/div[contains(@class, \'tip\')]/text()')
        kdas = html2.xpath('//td[@class="KDA Cell"]/span[contains(@class, \'KDARatio\')]/text()') #.replace(":1", "")
        danos = html2.xpath('//div[@class="ChampionDamage"]/text()') #replace(",", "")
        wards = html2.xpath('//span[@class="SightWard"]/text()')
        css = html2.xpath('//div[@class="CS"]/text()')
        
        #DEBUGMODE JOGADORES DA PARTIDA  
        '''
        for i in range(10):
            print("GameID: " + gameId  + 
              " SummonerId: " + nicks[i] +              
              " Campeão: " + campeoes[i] +
              " KDA: " + kdas[i].replace(":1", "")  + 
              " Dano: " + danos[i].replace(",", "")  + 
              " Wards: " + wards[i] + 
              " CS: " + css[i])
        '''
        
        for i in range(10):
            summonerId = findIdByName(nicks[i])
            campeaoNome = campeoes[i]
            kdaPartida = kdas[i].replace(":1", "").replace("Perfect", "10.00")
            danoPartida = danos[i].replace(",", "")
            wardsPartida = wards[i]
            csPartida = css[i]
            #print("INSERT INTO jogador_partida('gameId', 'summonerId', 'campeaoNome', 'kdaPartida', 'danoPartida', 'wardsPartida', 'csPartida') VALUES("
            #  +gameId+","+summonerId+",\""+campeaoNome+"\","+kdaPartida+","+danoPartida+","+wardsPartida+","+csPartida+");\n")
            #'''
            jogadoresPartidaSql = open(jogadoresPartidaPath, "a")
            jogadoresPartidaSql.write("INSERT INTO jogador_partida('gameId', 'summonerId', 'campeaoNome', 'kdaPartida', 'danoPartida', 'wardsPartida', 'csPartida') VALUES("
              +gameId+","+summonerId+",\""+campeaoNome+"\","+kdaPartida+","+danoPartida+","+wardsPartida+","+csPartida+");\n")
            jogadoresPartidaSql.close()
            #'''

In [5]:
nick = pd.read_csv(nicksPath)
nick.drop_duplicates(subset ="nick", 
                     keep = False, inplace = True)
nicks = nick['nick']

In [ ]:
for jogador in nicks:
    log = open(logPath, "a")
    logS = str(datetime.datetime.now())+" - "+jogador+"\n"
    print(logS, end='\r\r')
    log.write(logS)
    log.close()
    summonerId = getJogadorData(jogador.strip())
    #if(summonerId!=-1):
    #    getCampeoesJogadorData(summonerId)
    #    getPartidasJogadorData(summonerId)

In [3]:
#Adiciona Jogadores 
#bd = sqlite3.connect(bdPath) 
#jogadores = open(jogadoresPath, 'r').read()
#bd.executescript(jogadores)
#ids = pd.read_sql('SELECT (jogador.summonerId) FROM jogador', bd)
#bd.close()

In [51]:
#Apagar Arquivos de Teste
#if(os.path.isfile(logPath)):
#    os.remove(logPath)
#if(os.path.isfile(jogadoresPath)):
#    os.remove(jogadoresPath)
if(os.path.isfile(campeoesJogadorPath)):
    os.remove(campeoesJogadorPath)
if(os.path.isfile(partidasPath)):
    os.remove(partidasPath)
if(os.path.isfile(jogadoresPartidaPath)):
    os.remove(jogadoresPartidaPath)

In [8]:
#Captura dos campeoes da base de jogadores - sucesso
idss = ids['summonerId']
for idd in idss:
    logS = str(datetime.datetime.now())+" - "+str(idd)+"\n"
    print(logS, end='\r\r')
    getCampeoesJogadorData(str(idd))

2019-01-21 19:17:04.261341 - 400152
2019-01-21 19:17:04.607659 - 400163
2019-01-21 19:17:04.837022 - 400245
2019-01-21 19:17:05.153874 - 400258
2019-01-21 19:17:05.475383 - 400285
2019-01-21 19:17:05.840330 - 400378
2019-01-21 19:17:06.189660 - 400412
2019-01-21 19:17:06.535760 - 400442
2019-01-21 19:17:06.907486 - 400462
2019-01-21 19:17:07.323464 - 400463
2019-01-21 19:17:07.650566 - 400629
2019-01-21 19:17:08.006089 - 400656
2019-01-21 19:17:08.340769 - 400708
2019-01-21 19:17:08.710428 - 400784
2019-01-21 19:17:09.025721 - 400786
2019-01-21 19:17:09.387862 - 400789
2019-01-21 19:17:09.865415 - 400844
2019-01-21 19:17:10.194293 - 400858
2019-01-21 19:17:10.540753 - 400892
2019-01-21 19:17:10.838414 - 400972
2019-01-21 19:17:11.193412 - 400994
2019-01-21 19:17:11.515406 - 401013
2019-01-21 19:17:11.857649 - 401157
2019-01-21 19:17:12.184421 - 401170
2019-01-21 19:17:12.473822 - 401190
2019-01-21 19:17:12.837210 - 401197
2019-01-21 19:17:13.059258 - 401210
2019-01-21 19:17:13.410615 -

2019-01-21 19:18:38.109093 - 411597
2019-01-21 19:18:38.543683 - 411628
2019-01-21 19:18:38.898922 - 411635
2019-01-21 19:18:39.266212 - 411669
2019-01-21 19:18:39.636282 - 411680
2019-01-21 19:18:39.993484 - 411683
2019-01-21 19:18:40.329683 - 411740
2019-01-21 19:18:40.729441 - 411765
2019-01-21 19:18:41.097887 - 411766
2019-01-21 19:18:41.447263 - 411810
2019-01-21 19:18:41.825361 - 411814
2019-01-21 19:18:42.299250 - 411817
2019-01-21 19:18:42.686734 - 411873
2019-01-21 19:18:42.982688 - 411915
2019-01-21 19:18:43.387578 - 411936
2019-01-21 19:18:43.849229 - 411962
2019-01-21 19:18:44.199303 - 411972
2019-01-21 19:18:44.519741 - 412023
2019-01-21 19:18:44.889231 - 412047
2019-01-21 19:18:45.996678 - 412056
2019-01-21 19:18:46.389762 - 412124
2019-01-21 19:18:46.938194 - 412206
2019-01-21 19:18:47.366892 - 412239
2019-01-21 19:18:47.805268 - 412312
2019-01-21 19:18:48.885940 - 412323
2019-01-21 19:18:49.272787 - 412420
2019-01-21 19:18:49.619655 - 412568
2019-01-21 19:18:50.018370 -

2019-01-21 19:20:08.982761 - 431410
2019-01-21 19:20:09.388858 - 431456
2019-01-21 19:20:09.794470 - 431490
2019-01-21 19:20:10.451080 - 431633
2019-01-21 19:20:10.804378 - 432337
2019-01-21 19:20:11.169783 - 432510
2019-01-21 19:20:11.536888 - 432513
2019-01-21 19:20:11.887583 - 432770
2019-01-21 19:20:12.248446 - 432824
2019-01-21 19:20:12.564252 - 433011
2019-01-21 19:20:12.916779 - 433414
2019-01-21 19:20:13.342819 - 433661
2019-01-21 19:20:13.675939 - 434072
2019-01-21 19:20:14.103752 - 434171
2019-01-21 19:20:14.668752 - 434211
2019-01-21 19:20:15.065690 - 434527
2019-01-21 19:20:15.426756 - 434722
2019-01-21 19:20:15.933768 - 434776
2019-01-21 19:20:16.348808 - 434833
2019-01-21 19:20:16.729281 - 434923
2019-01-21 19:20:17.287909 - 435157
2019-01-21 19:20:17.650756 - 435226
2019-01-21 19:20:27.922659 - 435330
2019-01-21 19:20:28.250034 - 435455
2019-01-21 19:20:28.551834 - 435517
2019-01-21 19:20:28.909337 - 435531
2019-01-21 19:20:29.334327 - 435657
2019-01-21 19:20:29.694469 -

2019-01-21 19:21:57.349304 - 449287
2019-01-21 19:21:57.649469 - 449315
2019-01-21 19:21:57.996345 - 449387
2019-01-21 19:21:58.347091 - 449426
2019-01-21 19:21:58.683254 - 449694
2019-01-21 19:21:59.057835 - 449733
2019-01-21 19:21:59.395064 - 449761
2019-01-21 19:21:59.731709 - 449796
2019-01-21 19:22:00.035698 - 449841
2019-01-21 19:22:00.360586 - 449899
2019-01-21 19:22:00.728627 - 449946
2019-01-21 19:22:01.061272 - 450026
2019-01-21 19:22:01.446635 - 450053
2019-01-21 19:22:01.813738 - 450097
2019-01-21 19:22:02.239507 - 450118
2019-01-21 19:22:02.560262 - 450125
2019-01-21 19:22:02.902412 - 450231
2019-01-21 19:22:03.314254 - 450236
2019-01-21 19:22:03.651860 - 450252
2019-01-21 19:22:03.959935 - 450353
2019-01-21 19:22:04.289771 - 450368
2019-01-21 19:22:04.599810 - 450372
2019-01-21 19:22:04.942248 - 450403
2019-01-21 19:22:05.283991 - 450409
2019-01-21 19:22:05.610892 - 450501
2019-01-21 19:22:05.995171 - 450507
2019-01-21 19:22:06.332771 - 450564
2019-01-21 19:22:06.711534 -

2019-01-21 19:23:21.309802 - 459699
2019-01-21 19:23:21.645270 - 459793
2019-01-21 19:23:21.977248 - 459826
2019-01-21 19:23:22.356334 - 460063
2019-01-21 19:23:22.680772 - 460100
2019-01-21 19:23:23.040986 - 460202
2019-01-21 19:23:23.401586 - 460270
2019-01-21 19:23:23.727691 - 460358
2019-01-21 19:23:24.065824 - 460656
2019-01-21 19:23:24.399116 - 460680
2019-01-21 19:23:24.740957 - 460711
2019-01-21 19:23:25.137388 - 460712
2019-01-21 19:23:25.509633 - 460726
2019-01-21 19:23:25.860876 - 460732
2019-01-21 19:23:26.666264 - 460863
2019-01-21 19:23:26.958857 - 460897
2019-01-21 19:23:27.344606 - 460898
2019-01-21 19:23:27.679768 - 461045
2019-01-21 19:23:28.048025 - 461120
2019-01-21 19:23:28.418949 - 461147
2019-01-21 19:23:28.819774 - 461211
2019-01-21 19:23:29.343759 - 461489
2019-01-21 19:23:29.836985 - 461496
2019-01-21 19:23:30.153599 - 461571
2019-01-21 19:23:30.667133 - 461610
2019-01-21 19:23:31.029602 - 461625
2019-01-21 19:23:31.364685 - 461724
2019-01-21 19:23:31.695992 -

...

2019-01-21 21:53:04.494970 - 31390126
2019-01-21 21:53:04.844378 - 31390129
2019-01-21 21:53:05.161546 - 31390132
2019-01-21 21:53:05.534065 - 31390133
2019-01-21 21:53:05.967958 - 31390134
2019-01-21 21:53:06.393587 - 31390138
2019-01-21 21:53:06.783673 - 31390139
2019-01-21 21:53:07.135498 - 31390141
2019-01-21 21:53:07.490163 - 31400130
2019-01-21 21:53:07.897639 - 31400150
2019-01-21 21:53:08.251950 - 31410129
2019-01-21 21:53:08.689348 - 31410132
2019-01-21 21:53:09.018701 - 31410134
2019-01-21 21:53:09.414029 - 31410138
2019-01-21 21:53:09.733130 - 31410140
2019-01-21 21:53:10.106312 - 31410142
2019-01-21 21:53:10.453423 - 31410145
2019-01-21 21:53:10.812062 - 31422877
2019-01-21 21:53:11.204821 - 31432737
2019-01-21 21:53:11.563919 - 31440894
2019-01-21 21:53:11.948311 - 31442886
2019-01-21 21:53:12.321437 - 31452689
2019-01-21 21:53:12.693776 - 31460187
2019-01-21 21:53:13.079125 - 31462880
2019-01-21 21:53:13.461514 - 31463270
2019-01-21 21:53:13.851162 - 31491013
2019-01-21 2

2019-01-21 21:54:25.523546 - 33100129
2019-01-21 21:54:25.875715 - 33111733
2019-01-21 21:54:26.248489 - 33121303
2019-01-21 21:54:26.588922 - 33122301
2019-01-21 21:54:26.942233 - 33141646
2019-01-21 21:54:27.298361 - 33150352
2019-01-21 21:54:27.648403 - 33150941
2019-01-21 21:54:27.914645 - 33160568
2019-01-21 21:54:28.346326 - 33162146
2019-01-21 21:54:28.638147 - 33171749
2019-01-21 21:54:29.000293 - 33172371
2019-01-21 21:54:29.396295 - 33191013
2019-01-21 21:54:29.745675 - 33192357
2019-01-21 21:54:30.036187 - 33202237
2019-01-21 21:54:30.427269 - 33202322
2019-01-21 21:54:30.759581 - 33230264
2019-01-21 21:54:31.569773 - 33230346
2019-01-21 21:54:31.803717 - 33230557
2019-01-21 21:54:32.155739 - 33232031
2019-01-21 21:54:32.475984 - 33250830
2019-01-21 21:54:32.825783 - 33303211
2019-01-21 21:54:33.132742 - 33303537
2019-01-21 21:54:33.519763 - 33303750
2019-01-21 21:54:33.833569 - 33304202
2019-01-21 21:54:34.067794 - 33312205
2019-01-21 21:54:34.472003 - 33320717
2019-01-21 2

2019-01-21 21:55:44.154865 - 34680126
2019-01-21 21:55:44.483629 - 34680128
2019-01-21 21:55:44.842943 - 34712572
2019-01-21 21:55:46.410174 - 34713267
2019-01-21 21:55:46.825795 - 34724014
2019-01-21 21:55:47.266773 - 34724316
2019-01-21 21:55:47.681889 - 34730629
2019-01-21 21:55:48.106890 - 34731143
2019-01-21 21:55:48.544807 - 34732591
2019-01-21 21:55:48.935717 - 34740235
2019-01-21 21:55:49.202461 - 34740527
2019-01-21 21:55:49.457733 - 34740646
2019-01-21 21:55:49.922408 - 34741171
2019-01-21 21:55:50.367967 - 34741642
2019-01-21 21:55:50.758218 - 34744190
2019-01-21 21:55:51.138474 - 34750892
2019-01-21 21:55:51.535555 - 34751147
2019-01-21 21:55:51.932631 - 34751148
2019-01-21 21:55:52.307865 - 34751706
2019-01-21 21:55:52.608447 - 34754207
2019-01-21 21:55:52.961334 - 34760359
2019-01-21 21:55:53.268982 - 34761097
2019-01-21 21:55:53.986915 - 34761107
2019-01-21 21:55:54.377608 - 34761109
2019-01-21 21:55:54.726694 - 34770205
2019-01-21 21:55:55.101291 - 34770535
2019-01-21 2

2019-01-21 21:57:05.226177 - 36032844
2019-01-21 21:57:05.549977 - 36032915
2019-01-21 21:57:05.928938 - 36032931
2019-01-21 21:57:06.366919 - 36032939
2019-01-21 21:57:06.770158 - 36033000
2019-01-21 21:57:07.239405 - 36041768
2019-01-21 21:57:07.560649 - 36042772
2019-01-21 21:57:07.946857 - 36042775
2019-01-21 21:57:08.332490 - 36042874
2019-01-21 21:57:08.590261 - 36042938
2019-01-21 21:57:08.906824 - 36042957
2019-01-21 21:57:09.150803 - 36042989
2019-01-21 21:57:09.389218 - 36043064
2019-01-21 21:57:09.725454 - 36043126
2019-01-21 21:57:10.082515 - 36052630
2019-01-21 21:57:10.424426 - 36052651
2019-01-21 21:57:10.771934 - 36052667
2019-01-21 21:57:11.112811 - 36052749
2019-01-21 21:57:11.502774 - 36052757
2019-01-21 21:57:12.155538 - 36052811
2019-01-21 21:57:12.404351 - 36052876
2019-01-21 21:57:12.759076 - 36052978
2019-01-21 21:57:13.124508 - 36053091
2019-01-21 21:57:13.469562 - 36053129
2019-01-21 21:57:13.818855 - 36053145
2019-01-21 21:57:14.200714 - 36060545
2019-01-21 2

2019-01-21 21:58:23.714244 - 37343436
2019-01-21 21:58:24.209379 - 37350423
2019-01-21 21:58:24.544307 - 37352365
2019-01-21 21:58:24.903085 - 37353556
2019-01-21 21:58:25.258429 - 37354768
2019-01-21 21:58:25.643460 - 37361668
2019-01-21 21:58:26.033144 - 37362853
2019-01-21 21:58:26.416187 - 37372039
2019-01-21 21:58:26.802926 - 37373410
2019-01-21 21:58:27.141105 - 37392603
2019-01-21 21:58:27.477481 - 37396080
2019-01-21 21:58:27.822891 - 37402669
2019-01-21 21:58:28.197703 - 37411807
2019-01-21 21:58:28.536615 - 37414669
2019-01-21 21:58:28.925514 - 37423070
2019-01-21 21:58:29.259515 - 37440138
2019-01-21 21:58:29.589215 - 37440145
2019-01-21 21:58:29.992698 - 37450129
2019-01-21 21:58:30.336790 - 37470126
2019-01-21 21:58:30.701253 - 37470131
2019-01-21 21:58:31.036554 - 37482608
2019-01-21 21:58:31.436427 - 37491592
2019-01-21 21:58:31.865131 - 37501333
2019-01-21 21:58:32.230235 - 37533458
2019-01-21 21:58:32.575073 - 37553405
2019-01-21 21:58:32.962544 - 37570532
2019-01-21 2

In [4]:
#Captura das partidas da base de jogadores - sucesso
idss = ids['summonerId']
for idd in idss:  
    logS = str(datetime.datetime.now())+" - "+str(idd)+"\n"
    print(logS, end='\r\r')
    getPartidasJogadorData(str(idd))

2019-01-22 01:36:47.635283 - 400152
2019-01-22 01:36:54.722760 - 400163
2019-01-22 01:37:03.125872 - 400245
2019-01-22 01:37:10.048183 - 400258
2019-01-22 01:37:16.206226 - 400285
2019-01-22 01:37:35.379113 - 400378
2019-01-22 01:37:55.059024 - 400412
2019-01-22 01:38:06.577182 - 400442
2019-01-22 01:38:15.461634 - 400462
2019-01-22 01:38:34.697808 - 400463
2019-01-22 01:39:01.326342 - 400629
2019-01-22 01:39:08.371312 - 400656
2019-01-22 01:39:17.474516 - 400708
2019-01-22 01:39:39.537376 - 400784
2019-01-22 01:40:04.828888 - 400786
2019-01-22 01:40:19.430009 - 400789
2019-01-22 01:40:46.278190 - 400844
2019-01-22 01:41:09.765093 - 400858
2019-01-22 01:41:19.668276 - 400892
2019-01-22 01:41:43.467906 - 400972
2019-01-22 01:41:50.647875 - 400994
2019-01-22 01:42:09.398468 - 401013
2019-01-22 01:42:20.145244 - 401157
2019-01-22 01:42:33.271448 - 401170
2019-01-22 01:42:41.106158 - 401190
2019-01-22 01:43:06.863076 - 401197
2019-01-22 01:43:31.006375 - 401210
2019-01-22 01:43:38.191858 -

2019-01-22 02:33:20.350852 - 411597
2019-01-22 02:33:36.348261 - 411628
2019-01-22 02:33:44.338978 - 411635
2019-01-22 02:33:52.144163 - 411669
2019-01-22 02:34:11.014899 - 411680
2019-01-22 02:34:30.780781 - 411683
2019-01-22 02:34:37.800048 - 411740
2019-01-22 02:34:44.527970 - 411765
2019-01-22 02:35:03.742587 - 411766
2019-01-22 02:35:10.709229 - 411810
2019-01-22 02:35:17.489908 - 411814
2019-01-22 02:35:36.712949 - 411817
2019-01-22 02:35:43.711352 - 411873
2019-01-22 02:36:02.263709 - 411915
2019-01-22 02:36:09.130091 - 411936
2019-01-22 02:36:27.760044 - 411962
2019-01-22 02:36:34.545268 - 411972
2019-01-22 02:36:52.562214 - 412023
2019-01-22 02:36:58.630012 - 412047
2019-01-22 02:37:04.014956 - 412056
2019-01-22 02:37:10.976215 - 412124
2019-01-22 02:37:29.548938 - 412206
2019-01-22 02:37:37.106403 - 412239
2019-01-22 02:37:45.337130 - 412312
2019-01-22 02:37:55.658515 - 412323
2019-01-22 02:38:16.158014 - 412420
2019-01-22 02:38:35.206037 - 412568
2019-01-22 02:38:42.133143 -

2019-01-22 03:26:06.614481 - 431410
2019-01-22 03:26:31.386135 - 431456
2019-01-22 03:26:52.865520 - 431490
2019-01-22 03:26:59.945608 - 431633
2019-01-22 03:27:05.245787 - 432337
2019-01-22 03:27:23.929370 - 432510
2019-01-22 03:27:40.434749 - 432513
2019-01-22 03:27:46.679793 - 432770
2019-01-22 03:28:06.325707 - 432824
2019-01-22 03:28:21.431828 - 433011
2019-01-22 03:28:39.928403 - 433414
2019-01-22 03:28:58.855843 - 433661
2019-01-22 03:29:10.755546 - 434072
2019-01-22 03:29:29.816272 - 434171
2019-01-22 03:29:48.379579 - 434211
2019-01-22 03:30:07.832574 - 434527
2019-01-22 03:30:28.423194 - 434722
2019-01-22 03:30:47.302466 - 434776
2019-01-22 03:31:07.458545 - 434833
2019-01-22 03:31:30.111332 - 434923
2019-01-22 03:31:40.425883 - 435157
2019-01-22 03:31:59.135535 - 435226
2019-01-22 03:32:14.617463 - 435330
2019-01-22 03:32:32.922678 - 435455
2019-01-22 03:32:52.541227 - 435517
2019-01-22 03:32:59.651716 - 435531
2019-01-22 03:33:18.261725 - 435657
2019-01-22 03:33:24.946098 -

2019-01-22 04:20:19.541470 - 449287
2019-01-22 04:20:41.817210 - 449315
2019-01-22 04:20:50.055031 - 449387
2019-01-22 04:21:08.791115 - 449426
2019-01-22 04:21:15.765930 - 449694
2019-01-22 04:21:33.310976 - 449733
2019-01-22 04:21:53.038143 - 449761
2019-01-22 04:22:00.227041 - 449796
2019-01-22 04:22:06.989226 - 449841
2019-01-22 04:22:13.712179 - 449899
2019-01-22 04:22:20.794697 - 449946
2019-01-22 04:22:39.112359 - 450026
2019-01-22 04:22:46.390959 - 450053
2019-01-22 04:23:05.023903 - 450097
2019-01-22 04:23:23.226438 - 450118
2019-01-22 04:23:41.702036 - 450125
2019-01-22 04:23:48.639568 - 450231
2019-01-22 04:23:55.393558 - 450236
2019-01-22 04:24:13.223359 - 450252
2019-01-22 04:24:20.255082 - 450353
2019-01-22 04:24:39.896965 - 450368
2019-01-22 04:24:52.854251 - 450372
2019-01-22 04:24:59.747208 - 450403
2019-01-22 04:25:18.519306 - 450409
2019-01-22 04:25:35.371570 - 450501
2019-01-22 04:25:53.415975 - 450507
2019-01-22 04:25:58.884902 - 450564
2019-01-22 04:26:06.247988 -

2019-01-22 05:11:07.663544 - 459699
2019-01-22 05:11:27.141821 - 459793
2019-01-22 05:11:45.822856 - 459826
2019-01-22 05:12:04.266935 - 460063
2019-01-22 05:12:23.527222 - 460100
2019-01-22 05:12:28.923320 - 460202
2019-01-22 05:12:35.057607 - 460270
2019-01-22 05:12:53.628625 - 460358
2019-01-22 05:13:12.770006 - 460656
2019-01-22 05:13:19.621642 - 460680
2019-01-22 05:13:37.849210 - 460711
2019-01-22 05:13:56.772331 - 460712
2019-01-22 05:14:15.350320 - 460726
2019-01-22 05:14:34.023498 - 460732
2019-01-22 05:14:53.726383 - 460863
2019-01-22 05:15:00.373597 - 460897
2019-01-22 05:15:07.551448 - 460898
2019-01-22 05:15:22.600383 - 461045
2019-01-22 05:15:29.687080 - 461120
2019-01-22 05:15:37.886834 - 461147
2019-01-22 05:15:44.651002 - 461211
2019-01-22 05:16:03.517219 - 461489
2019-01-22 05:16:23.243382 - 461496
2019-01-22 05:16:49.557769 - 461571
2019-01-22 05:16:56.817488 - 461610
2019-01-22 05:17:15.565685 - 461625
2019-01-22 05:17:35.119424 - 461724
2019-01-22 05:17:46.116107 -

ParserError: Document is empty